<a href="https://colab.research.google.com/github/ESGcompetition/ipnyb/blob/choiyebin/%EA%B3%B5%EB%AA%A8%EC%A0%84_%EA%B0%80%EC%A4%91%EC%B9%98%ED%95%A8%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/package'
save_path = '/content/drive/MyDrive/Colab Notebooks/package' 

os.symlink(save_path, my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re

## opendartAPI Import

In [3]:
import OpenDartReader

api_key = 'ebd213d75beaeac7659874cda6ec4ec28394fdc3'
dart = OpenDartReader(api_key) 

# 데이터셋 생성

임원 데이터셋

In [4]:
ds_exe = dart.report('대신증권', '임원', 2022)

In [5]:
ds_exe.columns = ['접수번호', '법인구분', '고유번호', '법인명', '이름', '성별', '생년월일', '직위', '등기임원여부', '상근여부', '담당업무', '주요경력', '최대주주와의관계', '재직기간', '임기만료일']

In [6]:
ds_exe.head()

,접수번호,법인구분,고유번호,법인명,이름,성별,생년월일,직위,등기임원여부,상근여부,담당업무,주요경력,최대주주와의관계,재직기간,임기만료일
0,20230316001545,Y,00110893,대신증권,이어룡,여,1953년 09월,회장,사내이사,상근,-,상명여자사범대,1항 1호 나,2004.09.24~2022.12.31,2023년 03월 31일
1,20230316001545,Y,00110893,대신증권,송혁,남,1964년 04월,부사장,미등기,상근,리테일총괄,전북대 회계학\n 대신증권 비서실장\n 대신증권 비서/브랜드본부장\n 대신증권 WM...,-,2010.04.01~2022.12.31,2024년 03월 31일
2,20230316001545,Y,00110893,대신증권,길기모,남,1968년 12월,전무,미등기,상근,리스크관리부문장,서울대 경제학\n 메리츠종금증권 심사팀장\n 메리츠종금증권 리스크관리본부장,-,2019.04.01~2022.12.31,2023년 03월 31일
3,20230316001545,Y,00110893,대신증권,박성준,남,1973년 02월,전무,미등기,상근,IB부문장,홍익대 경영학\n 대신증권 금융주치의전략부장\n 대신증권 IB솔루션부장\n 대신증권...,-,2018.01.01~2022.12.31,2024년 03월 31일
4,20230316001545,Y,00110893,대신증권,이정화,남,1968년 11월,전무,미등기,상근,-,부경대 무역학\n 대신증권 마산지점장\n 대신증권 부전동지점장\n 대신증권 동래지점장,-,2016.01.01~2022.12.31,2023년 03월 31일


# 데이터 전처리

In [7]:
#대학, 전공 피처 생성
temp=ds_exe['주요경력'].str.split('\n')
colleges=temp.str.get(0)

colleges[colleges== 'University of Bath 사회과학대학원(석사)'] ='UniversityofBath 사회과학대학원(석사)'
colleges[colleges== 'Florida International University 경영대학원(박사)']='FloridaInternationalUniversity 경영대학원(박사)'
colleges[colleges== 'Ohio State University 정책대학원(박사)'] = 'OhioStateUniversity 정책대학원(박사)'

#대학, 전공으로 나누기
ttemp = colleges.str.split() 

#대학 피처 생성
ds_exe['대학교']=ttemp.str.get(0)

#전공 피처 생성
tttemp = ttemp.str.get(1)
tttemp=tttemp.replace(r'\([^)]*\)', '', regex=True)
ds_exe['전공']=tttemp

#사범대에 전공Nan 처리
ds_exe.fillna('교육학', inplace=True)

In [8]:
#근속연수 생성
temp = ds_exe['재직기간'].str.split('~\n|~')
firstY=temp.str.get(0)
secondY=temp.str.get(1)

firstY=firstY.apply(lambda x : x[:4])
secondY=secondY.apply(lambda x : x[:4])

ds_exe['근속연수']=secondY.astype('int')-firstY.astype('int')

In [26]:
#주요경력 전처리
def replace_career(x):
  temp = [string.split() for string in re.split(r'\n |\n', x)]
  return set(list(zip(*temp[1:]))[0])

ds_exe['주요경력요약'] = ds_exe['주요경력'].iloc[1:].apply(lambda x : replace_career(x))

#Nan 처리
ds_exe['주요경력요약'].iloc[0] = set(['대신증권'])

<ipython-input-26-36cc29a5502b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_exe['주요경력요약'].iloc[0] = set(['대신증권'])


In [27]:
ds_exe.head()

,접수번호,법인구분,고유번호,법인명,이름,성별,생년월일,직위,등기임원여부,상근여부,담당업무,주요경력,최대주주와의관계,재직기간,임기만료일,대학교,전공,근속연수,주요경력요약
0,20230316001545,Y,00110893,대신증권,이어룡,여,1953년 09월,회장,사내이사,상근,-,상명여자사범대,1항 1호 나,2004.09.24~2022.12.31,2023년 03월 31일,상명여자사범대,교육학,18,{대신증권}
1,20230316001545,Y,00110893,대신증권,송혁,남,1964년 04월,부사장,미등기,상근,리테일총괄,전북대 회계학\n 대신증권 비서실장\n 대신증권 비서/브랜드본부장\n 대신증권 WM...,-,2010.04.01~2022.12.31,2024년 03월 31일,전북대,회계학,12,{대신증권}
2,20230316001545,Y,00110893,대신증권,길기모,남,1968년 12월,전무,미등기,상근,리스크관리부문장,서울대 경제학\n 메리츠종금증권 심사팀장\n 메리츠종금증권 리스크관리본부장,-,2019.04.01~2022.12.31,2023년 03월 31일,서울대,경제학,3,{메리츠종금증권}
3,20230316001545,Y,00110893,대신증권,박성준,남,1973년 02월,전무,미등기,상근,IB부문장,홍익대 경영학\n 대신증권 금융주치의전략부장\n 대신증권 IB솔루션부장\n 대신증권...,-,2018.01.01~2022.12.31,2024년 03월 31일,홍익대,경영학,4,{대신증권}
4,20230316001545,Y,00110893,대신증권,이정화,남,1968년 11월,전무,미등기,상근,-,부경대 무역학\n 대신증권 마산지점장\n 대신증권 부전동지점장\n 대신증권 동래지점장,-,2016.01.01~2022.12.31,2023년 03월 31일,부경대,무역학,6,{대신증권}


# 가중치 함수

In [116]:
#학교 거리 계산
def cal_college(df1, df2, alpha = 0.7):
  #지역별로 나눔
  college_dict = {'서울경기':['상명여자사범대', '서울대', '홍익대', '성균관대', '연세대', '동국대', '중앙대', '한국외대', '세종대학교', '성균관대학교', '경희대', '서강대', '고려대', '건국대', '가천대'],
                   '광역시':['부경대', '경북대', '전남대', '조선대', '한국과학기술원', '가천대', '인하대', '동아대'],
                   '전북':['전북대', '원광대'],
                   '해외':['FloridaInternationalUniversity', 'OhioStateUniversity']}

  df1_key=0
  df2_key=0

  #키값 찾기
  for key, val in college_dict.items() :
    if df1['대학교'] in val :
      df1_key = key
    if df2['대학교'] in val :
      df2_key = key
          
  #k값 구하기
  if df1['대학교'] == df2['대학교'] :
    k=0
  elif df1_key == df2_key :
    k=1
  elif df1_key != df2_key :
    k=2

  return alpha ** k

전북: 전북대(전주), 원광대(익산) \
대전: 한국과학기술원 \
대구: 경북대 \
광주: 전남대, 조선대 \
부산: 부경대, 동아대 \
인천: 인하대 \
경기: 가천대 \

(시별로 나누면 같은 지역이 너무 많고 지역마다 학교도 적어서 일단 아래처럼 나눔) \
경기=>서울통합 \
광역시통합 \
전북

In [117]:
#학과 거리 계산
def cal_department(df1, df2, alpha = 0.7):
  department_dict={'경영경제':['회계학', '경제학', '경역학', '무역학', 'MBA', '경제통상학', '경영대학원', '산업정보학'],
                   '사회과학':['사회학', '행정학', '정치외교학'],
                   '법률':['법학', '정책대학원'],
                   '기타':['교육학', '인적자원개발학'],
                   '이공계':['기계공학', '응용통계학', '통계학']}

  df1_key=0
  df2_key=0

  #키값 찾기
  for key, val in department_dict.items() :
    if df1['전공'] in val :
      df1_key = key
    if df2['전공'] in val :
      df2_key = key
          
  #k값 구하기
  if df1['전공'] == df2['전공'] :
    k=0
  elif df1_key == df2_key :
    k=1
  elif df1_key != df2_key :
    k=2

  return alpha ** k

In [118]:
#직위 거리 계산
def cal_position(df1, df2, alpha = 0.7):
  position_dict={'회장':0, '대표이사':1, '부회장':2,'부사장':3, '전무':4, '상무': 5, '상무보':6, '이사대우부문장':7, '사외이사':99}

  #value값 찾기
  df1_value = position_dict[df1['직위']]
  df2_value = position_dict[df2['직위']]

  #k값 구하기
  if df1['직위'] == df2['직위'] :
    k=0
  elif df1_value == 99 or df2_value == 99 :
    k=2
  else :
    k=abs(df1_value - df2_value)

  return alpha ** k

대신증권 계급도 못찾겠음..

In [119]:
# 근속연수 거리 계산
def cal_work_years(df1, df2, alpha = 0.7):
  #k값 구하기
  if df1['근속연수'] == df2['근속연수'] :
    k=0
  else :
    if abs(df1['근속연수'] - df2['근속연수']) % 5 ==0:    #근속연수 차이 5년 기준으로 1씩 증가 (차이 1~5/6~10/11~15/...같은그룹)
      k= abs(df1['근속연수'] - df2['근속연수']) // 5 
    else:
      k= 1 + abs(df1['근속연수'] - df2['근속연수']) // 5 

  return alpha ** k  

http://money2.daishin.co.kr/company/recruit/job_introduce.shtml

http://www.educe.co.kr/company/jobinfo_view.php?sma_no=60&keyword=&search_txt2=

In [121]:
#담당업무 거리 계산
def cal_task(df1, df2, alpha = 0.7):
  task_dict={'IT계열': {'IT':['IT부문장'], '디지털':['디지털부문장'], '정보보호':['정보보호부문장']},
             '본사영업':{'Wolesale':['Wholesale부문장', 'Trading부문장'], 
                     'IB':['IB부문장', 'IPO담당',  'PF부문장', 'PF1본부장'], 
                     '영업':['영업부장']},
             '본사관리':{'리테일':['리테일총괄', 'Club1962센터장','WM추진부문장'],
                     '리스크':['리스크관리부문장'] ,
                     '경영':['경영전략총괄', '전략지원부문장', '경영지원부문장', '경영기획부문장'], 
                     '업무':['업무총괄'], 
                     '금융':['기업금융담당'], 
                     '법률':['준법감시인\n 준법지원부문장'], 
                     '감사':['감사위원장', '감사부문장', '감사위원'], 
                     '언론홍보':['홍보부문장', 'Coverage본부장']},
             '리서치': {'리서치센터':['Research Center장']},
             '고객':{'고객자산':['고객자산부문장', '고객자산부문 부부문장'],
                   'WM':['서부WM부문장', '재경1WM부문장', '재경2WM부문장', '동부WM부문장'],
                   '프라이빗':['프라이빗부문장,\n 나인원프라이빗라운지장'], 
                   '소비자보호':['금융소비자보호부문장\n(COO)']},
             '기타':{'비서':['비서실장'], '공란':['-']}}

  df1_outerKey=0
  df2_outerKey=0

    #키값 찾기
  for outerKey in task_dict.keys() :
    for key, val in task_dict[outerKey].items() :
      if df1['담당업무'] in val :
        df1_outerKey = outerKey
        df1_innerKey = key
      if df2['담당업무'] in val :
        df2_outerKey = outerKey
        df2_innerKey = key
          
          
  #k값 구하기
  if df1['담당업무'] == df2['담당업무'] : #같은업무
    k=0
  elif df1_outerKey == df2_outerKey and df1_innerKey == df2_innerKey: #상위하위 혹은 비슷한업무
    k=1
  elif df1_outerKey == df2_outerKey : #같은계열업무
    k=2
  elif df1_outerKey != df2_outerKey :
    k=3 #다른계열업무

  return alpha ** k

In [136]:
#경력 거리 계산
def cal_career(df1, df2, alpha = 0.7):
  career_dict={0:['대신증권', 'WM추진부문장', '재경1WM부문장'], #대신증권
               1:['대신에프앤아이', '대신투자신탁운용', '대신에이엠씨', '대신자산운용', '대신저축은행'], #대신증권 자회사
               2:['메리츠종금증권', '미래에셋증권', 'KTB투자증권', 'SC제일은행', '메리츠증권', '하나금융투자', 'IBK투자증권', 'DB금융투자', 'NH투자증권', 'KB증권'], #타증권사
               3:['現)금융위원회', '감사원', '대검찰청', '現)법무법인', '서울북부지방검찰청', '중앙대', '한국회계정보학회', '現)중앙대', '서울시립대', '한국조세연구원', '現)서울시립대', '국가청렴위원회', '제58대', '現)재단법인'] #기타
                }
  
  #경력 여부 빈리스트 생성
  df1_list = [0,0,0,0] #0번인덱스: 대신증권, 1번인덱스: 대신증권 자회사, 2번인덱스: 타증권사, 3번인덱스: 기타
  df2_list = [0,0,0,0]

  #리스트에 각 업종에 종사했었으면 1, 아니면 0
  for career in df1['주요경력요약'] :
    for key, val in career_dict.items() :
      if career in val :
        df1_list[key]=1

  for career in df2['주요경력요약'] :
    for key, val in career_dict.items() :
      if career in val :
        df2_list[key]=1
  
  #범주 나누기
  if sum(df1_list) != 1: #대신+자회사 (현재 데이터에선 자회사있으면 대신증권이랑 항상 같이나옴, 나머지는 같이 나오지 않음)
    d1 = 1
  else:
    d1 = df1_list.index(1) #0 ->대신, 1->자회사, 2->타증권사, 3->기타 //1인 인덱스 찾기

  if sum(df2_list) != 1:
    d2 = 1
  else:
    d2 = df2_list.index(1) #0 ->대신, 1->자회사, 2->타증권사, 3->기타

  #k값 구하기
  if d1 == d2 : #같으면 0
    k=0
  elif d1==3 or d2==3: #같지 않을 때, 기타는 모두와 거리 3
    k=3
  elif d1==2 or d2==2: #증권사는 기타빼고 모두와 거리 2
    k=2
  elif (d1==0 and d2==1) or (d2==0 and d1==1): #대신, 자회사 끼리 거리는 1
    k=1

  return alpha ** k

0. 대신증권에만 있었던 사람들
1. 대신증권 자회사
2. 같은 금융업
3. 다른업종
4. 대신증권&자회사

In [75]:
#가중치 함수
def weight_sum(df1, df2):
  wt_college = cal_college(df1, df2)
  wt_department = cal_department(df1, df2)
  wt_position = cal_position(df1, df2)
  wt_task = cal_task(df1, df2)
  wt_work_years = cal_work_years(df1, df2)
  wt_career = cal_career(df1, df2)
  return wt_college + wt_department + wt_position + wt_task + wt_work_years + wt_career

In [102]:
cal_college(ds_exe.iloc[1], ds_exe.iloc[2])

0.48999999999999994

In [103]:
cal_department(ds_exe.iloc[1], ds_exe.iloc[2])

0.7

In [104]:
cal_position(ds_exe.iloc[1], ds_exe.iloc[2])

0.7

In [105]:
cal_work_years(ds_exe.iloc[0], ds_exe.iloc[3])

0.3429999999999999

In [123]:
cal_task(ds_exe.iloc[33], ds_exe.iloc[35])

0.7

In [137]:
cal_career(ds_exe.iloc[0], ds_exe.iloc[23])

0.48999999999999994

In [140]:
cal_career(ds_exe.iloc[37], ds_exe.iloc[7])

0.3429999999999999

In [134]:
ds_exe['주요경력요약']

0                               {대신증권}
1                               {대신증권}
2                            {메리츠종금증권}
3                               {대신증권}
4                               {대신증권}
5                               {대신증권}
6                               {대신증권}
7                      {대신에프앤아이, 대신증권}
8                               {대신증권}
9                               {대신증권}
10                              {대신증권}
11                    {대신증권, 대신투자신탁운용}
12                              {대신증권}
13                              {대신증권}
14                              {대신증권}
15      {대신에이엠씨, 대신증권, 대신저축은행, 대신자산운용}
16                              {대신증권}
17                              {대신증권}
18                     {WM추진부문장, 대신증권}
19                    {재경1WM부문장, 대신증권}
20                              {대신증권}
21                              {대신증권}
22                      {대신증권, 대신저축은행}
23           {미래에셋증권, SC제일은행, KTB투자증권}
24                              {대신증권}
25                       

# 가중치 행렬

In [141]:
# 인물 가중치행렬
col = []
row = []
for i in range(len(ds_exe)):
    col.append(ds_exe.loc[i, '이름'])
    row.append(ds_exe.loc[i, '이름'])


weight_matrix = pd.DataFrame(columns=col, index=row)

In [142]:
for i in range(0, len(ds_exe)):
    for j in range(0, len(ds_exe)):
      weight_matrix.iloc[i, j] = weight_sum(ds_exe.iloc[i], ds_exe.iloc[j])

In [143]:
weight_matrix

,이어룡,송혁,길기모,박성준,이정화,권택현,정연규,김호중,정연우,이재우,...,김봉진,김병철,김상혁,이성영,정회민,이창세,김창수,원윤희,김성호,김범철
이어룡,6.0,3.156,2.6061,3.1161,3.5631,2.9061,2.9061,2.7132,3.1161,3.1161,...,2.680749,2.709,2.170749,2.855454,2.645454,3.366,2.3961,3.0531,2.6061,2.919
송혁,3.156,6.0,3.36,3.513,3.723,3.513,4.023,3.423,3.513,3.513,...,3.009,2.793,2.499,3.2631,3.1161,2.646,2.856,2.499,2.646,3.66
길기모,2.6061,3.36,6.0,3.723,3.723,3.513,3.723,3.933,4.023,4.023,...,3.003,3.213,3.513,3.423,2.856,3.366,3.213,2.856,3.213,3.57
박성준,3.1161,3.513,3.723,6.0,4.023,5.19,4.323,3.933,4.743,5.19,...,4.17,3.366,3.36,3.576,3.366,3.366,2.856,2.856,3.066,4.143
이정화,3.5631,3.723,3.723,4.023,6.0,4.233,4.443,4.233,4.023,4.023,...,3.723,2.856,3.003,3.876,3.156,3.513,3.066,3.513,2.646,3.723
권택현,2.9061,3.513,3.513,5.19,4.233,6.0,4.533,3.723,4.533,4.68,...,4.38,3.156,3.57,3.366,3.366,3.156,2.856,2.856,2.856,3.933
정연규,2.9061,4.023,3.723,4.323,4.443,4.533,6.0,3.933,4.023,4.023,...,3.723,3.156,3.213,3.576,3.366,3.156,3.066,2.856,2.856,3.423
김호중,2.7132,3.423,3.933,3.933,4.233,3.723,3.933,6.0,3.933,3.933,...,3.213,3.366,3.003,3.786,3.066,3.066,3.066,2.856,3.066,3.933
정연우,3.1161,3.513,4.023,4.743,4.023,4.533,4.023,3.933,6.0,5.043,...,4.023,3.066,3.003,3.576,3.366,3.066,2.856,2.856,3.066,4.143
이재우,3.1161,3.513,4.023,5.19,4.023,4.68,4.023,3.933,5.043,6.0,...,4.17,3.066,3.15,3.576,3.366,3.066,2.856,2.856,3.066,4.143


# 기타test

In [35]:
ds_2021 = dart.report('대신증권', '임원', 2021)
ds_2021.columns = ['접수번호', '법인구분', '고유번호', '법인명', '이름', '성별', '생년월일', '직위', '등기임원여부', '상근여부', '담당업무', '주요경력', '최대주주와의관계', '재직기간', '임기만료일']

ds_2021.info()
#주요경력 전처리
def replace_career(x):
  temp = [string.split() for string in re.split(r'\n |\n', x)]
  return set(list(zip(*temp[1:]))[0])

ds_2021['주요경력요약'] = ds_2021['주요경력'].iloc[1:].apply(lambda x : replace_career(x))

#Nan 처리
ds_2021['주요경력요약'].iloc[0] = set(['대신증권'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   접수번호      38 non-null     object
 1   법인구분      38 non-null     object
 2   고유번호      38 non-null     object
 3   법인명       38 non-null     object
 4   이름        38 non-null     object
 5   성별        38 non-null     object
 6   생년월일      38 non-null     object
 7   직위        38 non-null     object
 8   등기임원여부    38 non-null     object
 9   상근여부      38 non-null     object
 10  담당업무      38 non-null     object
 11  주요경력      38 non-null     object
 12  최대주주와의관계  38 non-null     object
 13  재직기간      38 non-null     object
 14  임기만료일     38 non-null     object
dtypes: object(15)
memory usage: 4.6+ KB


In [34]:
ds_2021.head()

,접수번호,법인구분,고유번호,법인명,이름,성별,생년월일,직위,등기임원여부,상근여부,담당업무,주요경력,최대주주와의관계,재직기간,임기만료일,주요경력요약
0,20220516001812,Y,00110893,대신증권,이어룡,여,1953년 09월,회장,사내이사,상근,-,상명여자사범대,1항 1호 나,2004.09.24~2021.12.31,2023년 03월 31일,{대신증권}
1,20220516001812,Y,00110893,대신증권,송혁,남,1964년 04월,부사장,미등기,상근,리테일총괄,전북대 회계학\n 대신증권 인수공모2부\n 대신증권 비서실장\n 대신증권 비서/브랜...,-,2010.04.01~2021.12.31,2023년 03월 31일,{대신증권}
2,20220516001812,Y,00110893,대신증권,길기모,남,1968년 12월,전무,미등기,상근,리스크관리부문장,서울대 경제학\n 메리츠종금증권 심사팀장\n 메리츠종금증권 리스크관리본부장,-,2019.04.01~2021.12.31,2023년 03월 31일,{메리츠종금증권}
3,20220516001812,Y,00110893,대신증권,유창범,남,1968년 04월,전무,미등기,상근,Sales&Trading총괄,서울대 행정학(석사)\n Bank of America 서울지점 지점장\n 메리츠증권...,-,2021.03.02~2021.12.31,2023년 03월 31일,"{메리츠증권, Bank}"
4,20220516001812,Y,00110893,대신증권,박성준,남,1973년 02월,전무,미등기,상근,IB부문장,홍익대 경영학\n 대신증권 금융주치의전략부장\n 대신증권 IB솔루션부장\n 대신증권...,-,2018.01.01~2021.12.31,2023년 03월 31일,{대신증권}


In [33]:
print(ds_2021['주요경력요약'])

0                               {대신증권}
1                               {대신증권}
2                            {메리츠종금증권}
3                        {메리츠증권, Bank}
4                               {대신증권}
5                               {대신증권}
6                               {대신증권}
7                      {대신에프앤아이, 대신증권}
8                               {대신증권}
9                               {대신증권}
10                     {대신에프앤아이, 대신증권}
11                    {대신증권, 대신투자신탁운용}
12                              {대신증권}
13                              {대신증권}
14                              {대신증권}
15                              {대신증권}
16                              {대신증권}
17                              {대신증권}
18      {대신에이엠씨, 대신증권, 대신저축은행, 대신자산운용}
19                              {대신증권}
20                              {대신증권}
21                              {대신증권}
22                      {대신증권, 대신저축은행}
23                              {대신증권}
24                              {대신증권}
25                       

In [36]:
ds_2020 = dart.report('대신증권', '임원', 2020)
ds_2020.columns = ['접수번호', '법인구분', '고유번호', '법인명', '이름', '성별', '생년월일', '직위', '등기임원여부', '상근여부', '담당업무', '주요경력', '최대주주와의관계', '재직기간', '임기만료일']

ds_2020.info()
#주요경력 전처리
def replace_career(x):
  temp = [string.split() for string in re.split(r'\n |\n', x)]
  return set(list(zip(*temp[1:]))[0])

ds_2020['주요경력요약'] = ds_2020['주요경력'].iloc[1:].apply(lambda x : replace_career(x))

#Nan 처리
ds_2020['주요경력요약'].iloc[0] = set(['대신증권'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   접수번호      36 non-null     object
 1   법인구분      36 non-null     object
 2   고유번호      36 non-null     object
 3   법인명       36 non-null     object
 4   이름        36 non-null     object
 5   성별        36 non-null     object
 6   생년월일      36 non-null     object
 7   직위        36 non-null     object
 8   등기임원여부    36 non-null     object
 9   상근여부      36 non-null     object
 10  담당업무      36 non-null     object
 11  주요경력      36 non-null     object
 12  최대주주와의관계  36 non-null     object
 13  재직기간      36 non-null     object
 14  임기만료일     36 non-null     object
dtypes: object(15)
memory usage: 4.3+ KB


In [37]:
print(ds_2020['주요경력요약'])

0                                      {대신증권}
1                          {대신증권, 대신증권WM사업단장}
2                                   {메리츠종금증권}
3                          {대신증권브랜드전략실, 대신증권}
4     {대신증권기획실장, 대신증권, 대신증권영업기획부장, 대신증권인재전략부}
5                                      {대신증권}
6                             {대신에프앤아이, 대신증권}
7                                      {대신증권}
8                                      {대신증권}
9                                      {대신증권}
10                                     {대신증권}
11                           {대신증권, 대신투자신탁운용}
12                            {대신에프앤아이, 대신증권}
13                                     {대신증권}
14                                     {대신증권}
15                                     {대신증권}
16                                     {대신증권}
17                                     {대신증권}
18                                     {대신증권}
19                                     {대신증권}
20             {대신에이엠씨, 대신증권, 대신저축은행, 대신자산운용}
21                           {이사대우

In [38]:
ds_2020.head(50)

,접수번호,법인구분,고유번호,법인명,이름,성별,생년월일,직위,등기임원여부,상근여부,담당업무,주요경력,최대주주와의관계,재직기간,임기만료일,주요경력요약
0,20210311001204,Y,00110893,대신증권,이어룡,여,1953년 09월,회장,등기임원,상근,-,상명여자사범대,1항 1호 나,2004.09.24~\n2020.12.31,2021년 03월 19일,{대신증권}
1,20210311001204,Y,00110893,대신증권,신인식,남,1962년 01월,전무,미등기임원,상근,-,성균관대 회계학\n대신증권 무역센터지점장\n 대신증권 강남선릉센터 이사대우센터장\n...,-,2013.04.01~\n2020.12.31,2021년 03월 19일,"{대신증권, 대신증권WM사업단장}"
2,20210311001204,Y,00110893,대신증권,길기모,남,1968년 12월,전무,미등기임원,상근,리스크관리부문장,서울대 경제학\n메리츠종금증권 심사팀장\n메리츠종금증권 리스크관리본부장,-,2019.04.01~\n2020.12.31,2021년 03월 31일,{메리츠종금증권}
3,20210311001204,Y,00110893,대신증권,조경순,남,1964년 01월,전무,미등기임원,상근,대외협력담당,연세대 경제학(석사)\n대신증권 홍보실 이사대우실장\n대신증권브랜드전략실 이사대우실...,-,2014.01.01~\n2020.12.31,2021년 03월 19일,"{대신증권브랜드전략실, 대신증권}"
4,20210311001204,Y,00110893,대신증권,홍대한,남,1963년 05월,전무,미등기임원,상근,준법감시인\n/준법지원부문장,서강대 경제학\n대신증권영업기획부장\n대신증권기획실장\n대신증권인재전략부 이사대우부...,-,2014.01.01~\n2020.12.31,2021년 12월 31일,"{대신증권기획실장, 대신증권, 대신증권영업기획부장, 대신증권인재전략부}"
5,20210311001204,Y,00110893,대신증권,이정화,남,1968년 11월,전무,미등기임원,상근,동부WM본부장,부경대 무역학\n대신증권 마산지점장\n대신증권 부전동지점장\n대신증권 동래지점장,-,2016.01.01~\n2020.12.31,2022년 03월 31일,{대신증권}
6,20210311001204,Y,00110893,대신증권,진승욱,남,1968년 08월,전무,미등기임원,상근,경영기획본부장,한양대 경영학\n대신증권 Global사업부장\n대신에프앤아이 경영기획본부 부본부장\...,-,2018.08.16~\n2020.12.31,2022년 03월 31일,"{대신에프앤아이, 대신증권}"
7,20210311001204,Y,00110893,대신증권,권택현,남,1972년 02월,전무,미등기임원,상근,PF부문장,경북대 경영학\n대신증권 SF부 팀장\n대신증권 프로젝트금융본부 이사대우본부장,-,2018.01.01~\n2020.12.31,2022년 03월 31일,{대신증권}
8,20210311001204,Y,00110893,대신증권,박성준,남,1973년 02월,전무,미등기임원,상근,IB부문장,홍익대 경영학\n대신증권 금융주치의전략부장\n대신증권 IB솔루션부장\n대신증권 IB...,-,2018.01.01~\n2020.12.31,2022년 03월 31일,{대신증권}
9,20210311001204,Y,00110893,대신증권,정연규,남,1967년 11월,전무,미등기임원,상근,서부WM본부장,전남대 회계학\n대신증권 목포지점장\n대신증권 광주지점장\n대신증권 광주센터장,-,2018.01.01~\n2020.12.31,2022년 03월 31일,{대신증권}
